In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("drug_consumption.csv")

# Using The Drugs Wheel (UK & Ireland DrugWatch) @ https://www.thedrugswheel.com/
dissociatives = ['ketamine']
depressants = ['alcohol', 'benzos']
opioids = ['heroin', 'methadone']
cannabinoids = ['cannabis']
stimulants = ['amphet', 'caff', 'coke', 'crack', 'nicotine']
empathogens = ['ecstasy']
psychedelics = ['lsd', 'mushrooms']

# seperate to include VSA & Amyl Nitrate
inhalants = ['amyl', 'vsa']

# EXCLUDE Chocolate & Legal Highs, goal is to predict for potential harm reduction planning/diagnostics
# use Principal Component Analysis (PCA) to create new features for the drug classes